# 03_transformations

transform and aggregate the data for the use cases

## Ideas:
* PCA on the high correlated static data for reducing dimensionality
* create new features for geolocations

* aggregating on geolocations + timeframes
* aggregating on ships?

* merging static and position data


## import packages

In [5]:
import sys
import os

import yaml
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from functions.utils import get_folder
from functions.data_loads import get_columns, load_data
from functions.data_cleaning import SimpleOutlierMask, create_mercator

pd.set_option('display.float_format', '{:.2f}'.format)

curr_folder = get_folder()
parent_fold = curr_folder.parent
with open(parent_fold / "connections.yml", encoding='utf-8') as file:
    db_config = yaml.safe_load(file)
    
with open(parent_fold / "configurations" / "data_loads.yaml", encoding='utf-8') as file:
    column_conf = yaml.safe_load(file)
    
engine = create_engine(f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:{db_config["port"]}/{db_config["db"]}')

dates = ["01.05.2022","05.05.2022"]

## load the data from the data bases and clean the data

In [6]:
position_columns = get_columns(engine,'tbl_positionnew',column_conf['position_unrel_cols'])

data = load_data(engine,'tbl_positionnew',position_columns)

SOM = SimpleOutlierMask()
# auskommentieren wenn nicht komplett neu trainiert werden soll
SOM.fit(data['speedoverground'])
#SOM.load(parent_fold / "modelle" / "simple_outlier_mask.pckl")
data['sog'] = SOM.transform(data['speedoverground'])
SOM.save(parent_fold / "modelle" / "simple_outlier_mask.pckl")
data.drop(['speedoverground'],axis = 1, inplace=True)
data.head()

,aisclass,courseoverground,latitude,longitude,messagetype,mmsi,msgtime,navigationalstatus,rateofturn,trueheading,sog
0,A,279.00,60.15,5.39,1,257072700,2022-04-07 17:00:00,0,0.00,226.00,0.00
1,A,0.00,58.15,8.03,1,258012180,2022-04-07 17:00:00,15,0.00,0.00,0.20
2,A,249.00,65.99,12.26,3,258489000,2022-04-07 17:00:00,5,0.00,206.00,0.00
3,A,179.50,68.36,16.01,3,257837000,2022-04-07 16:50:00,0,-5.00,174.00,13.90
4,A,206.50,63.22,4.32,1,311000548,2022-04-07 17:00:00,0,11.00,205.00,14.90


## perform the transformations
Group stage ist:
latitude, longitude, time

* round the latitude
* transform to mercator type
* extract import dates from msgtime 
  * date
  * day of week
  * hour
* save in new table in database with these new features and only the important other features
  * important features: [mmsi, sog, trueheading, rateofturn, courseoverground, lat, lon ( mercator), msggtime, date, dow, hod]

In [7]:
data[['lat','lon']] = data[['latitude','longitude']].round(decimals=1)
data = create_mercator(data)

data['msgtime'] = pd.to_datetime(data['msgtime'])
data['date'] = data['msgtime'].dt.date
data['dow'] = data['msgtime'].dt.dayofweek
data['hod'] = data['msgtime'].dt.hour
data.head()

C:\Users\danie\OneDrive\Dokumente\_Programmierung\python_workspaces\Uni_Master\ais_analysis\experiments\functions\data_cleaning.py:62: RuntimeWarning: invalid value encountered in double_scalars
  scale = x/lon


,aisclass,courseoverground,latitude,longitude,messagetype,mmsi,msgtime,navigationalstatus,rateofturn,trueheading,sog,lat,lon,lat_merc,lon_merc,date,dow,hod
0,A,279.00,60.15,5.39,1,257072700,2022-04-07 17:00:00,0,0.00,226.00,0.00,60.10,5.40,601125.25,8422035.52,2022-04-07,3,17
1,A,0.00,58.15,8.03,1,258012180,2022-04-07 17:00:00,15,0.00,0.00,0.20,58.10,8.00,890555.93,7988353.82,2022-04-07,3,17
2,A,249.00,65.99,12.26,3,258489000,2022-04-07 17:00:00,5,0.00,206.00,0.00,66.00,12.30,1369229.74,9876845.90,2022-04-07,3,17
3,A,179.50,68.36,16.01,3,257837000,2022-04-07 16:50:00,0,-5.00,174.00,13.90,68.40,16.00,1781111.85,10566902.74,2022-04-07,3,16
4,A,206.50,63.22,4.32,1,311000548,2022-04-07 17:00:00,0,11.00,205.00,14.90,63.20,4.30,478673.81,9149460.21,2022-04-07,3,17


In [8]:
# save the data in database
data = data[['lat_merc','lon_merc','msgtime','date','dow','hod','mmsi','aisclass','sog']]
data.to_sql("tbl_position_attr",engine,if_exists='replace', index=False)

876

## transform data features:
* count number of unique ships(mmsi) in area
* avg speed in area
* avg rateofturn
* avg trueheading?
* avg courseoverground


                                                        'trueheading':['mean','min','max'],
                                                        'rateofturn':['mean','min','max'],
                                                        'courseoverground':['mean','min','max'],

In [9]:
position_columns = get_columns(engine,'tbl_position_attr',column_conf['position_unrel_cols'])

data = load_data(engine,'tbl_position_attr',position_columns)

In [10]:
data.head()

,aisclass,date,dow,hod,lat_merc,lon_merc,mmsi,msgtime,sog
0,A,2022-04-10,6,18,2293181.51,11003930.41,257129600,2022-04-10 18:30:00,10.10
1,A,2022-04-07,3,17,1992618.89,11068715.66,258797000,2022-04-07 17:00:00,1.20
2,A,2022-04-07,3,0,1168854.65,8267349.13,257067640,2022-04-07 00:00:00,19.80
3,B,2022-04-10,6,16,957347.62,9273988.17,257814700,2022-04-10 16:00:00,0.05
4,A,2022-04-07,3,17,1491681.18,10358420.52,259987000,2022-04-07 17:00:00,16.50


In [11]:
data_grouped = data.groupby(['lat_merc','lon_merc','msgtime','date','dow','hod']).agg({'mmsi':pd.Series.nunique,'aisclass':'count',
                                                        'sog':['mean','min','max','std'],
                                                       })
data_grouped.columns = data_grouped.columns.get_level_values(0) + '_' +  data_grouped.columns.get_level_values(1)
data_grouped.reset_index(inplace=True)
print("new DF has Shape: ",data_grouped.shape, " Old Df has Shape:", data.shape)
data_grouped.head()

# add additionalfeatures

new DF has Shape:  (6206938, 12)  Old Df has Shape: (17602862, 9)


,lat_merc,lon_merc,msgtime,date,dow,hod,mmsi_nunique,aisclass_count,sog_mean,sog_min,sog_max,sog_std
0,-1157722.70,11167068.18,2022-06-03 00:50:00,2022-06-03,4,0,1,1,8.60,8.60,8.60,NaN
1,-1124326.86,11233438.68,2022-06-03 02:30:00,2022-06-03,4,2,1,1,7.10,7.10,7.10,NaN
2,-1035271.26,11334231.57,2022-06-03 04:10:00,2022-06-03,4,4,1,1,9.00,9.00,9.00,NaN
3,-1024139.32,11300466.67,2022-06-03 03:30:00,2022-06-03,4,3,1,1,9.10,9.10,9.10,NaN
4,-1024139.32,11300466.67,2022-06-08 08:50:00,2022-06-08,2,8,1,2,4.47,4.27,4.68,0.28


In [18]:
# get mean values over last hour
data_grouped = data_grouped.sort_values(['lat_merc','lon_merc','msgtime'], ascending=True)
data_member = data_grouped[['lat_merc','lon_merc','mmsi_nunique','aisclass_count','sog_mean','sog_min','sog_max','sog_std']].groupby(['lat_merc','lon_merc'], observed=True).rolling(6).mean().reset_index().sort_index()
data_member.head()

,lat_merc,lon_merc,level_2,mmsi_nunique,aisclass_count,sog_mean,sog_min,sog_max,sog_std
0,-1157722.70,11167068.18,0,NaN,NaN,NaN,NaN,NaN,NaN
1,-1124326.86,11233438.68,1,NaN,NaN,NaN,NaN,NaN,NaN
2,-1035271.26,11334231.57,2,NaN,NaN,NaN,NaN,NaN,NaN
3,-1024139.32,11300466.67,3,NaN,NaN,NaN,NaN,NaN,NaN
4,-1024139.32,11300466.67,4,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
data_member.rename({'aisclass_count':'aisclass_count_l_hour','mmsi_nunique':'mmsi_nunique_l_hour','sog_mean':'sog_mean_l_hour','sog_min':'sog_min_l_hour','sog_max':'sog_max_l_hour','sog_std':'sog_std_l_hour'}, axis=1,inplace=True)
#data_member.drop("level_2",axis=1,inplace=True)
data_merged = pd.merge(data_grouped, data_member, left_index=True, right_index=True)
data_merged.fillna(0, inplace = True)
print("new DF has Shape: ",data_merged.shape, " Old Df has Shape:", data.shape)

new DF has Shape:  (6206938, 20)  Old Df has Shape: (17602862, 9)


In [30]:
data_merged.head()

,lat_merc_x,lon_merc_x,msgtime,date,dow,hod,mmsi_nunique,aisclass_count,sog_mean,sog_min,sog_max,sog_std,lat_merc_y,lon_merc_y,mmsi_nunique_l_hour,aisclass_count_l_hour,sog_mean_l_hour,sog_min_l_hour,sog_max_l_hour,sog_std_l_hour
0,-1157722.70,11167068.18,2022-06-03 00:50:00,2022-06-03,4,0,1,1,8.60,8.60,8.60,0.00,-1157722.70,11167068.18,0.00,0.00,0.00,0.00,0.00,0.00
1,-1124326.86,11233438.68,2022-06-03 02:30:00,2022-06-03,4,2,1,1,7.10,7.10,7.10,0.00,-1124326.86,11233438.68,0.00,0.00,0.00,0.00,0.00,0.00
2,-1035271.26,11334231.57,2022-06-03 04:10:00,2022-06-03,4,4,1,1,9.00,9.00,9.00,0.00,-1035271.26,11334231.57,0.00,0.00,0.00,0.00,0.00,0.00
3,-1024139.32,11300466.67,2022-06-03 03:30:00,2022-06-03,4,3,1,1,9.10,9.10,9.10,0.00,-1024139.32,11300466.67,0.00,0.00,0.00,0.00,0.00,0.00
4,-1024139.32,11300466.67,2022-06-08 08:50:00,2022-06-08,2,8,1,2,4.47,4.27,4.68,0.28,-1024139.32,11300466.67,0.00,0.00,0.00,0.00,0.00,0.00


In [31]:
data_merged.to_sql("tbl_positionagg",engine,if_exists='replace', index=False)

938